In [1]:
## Install tesserocr: https://pypi.org/project/tesserocr/
## Install pdf2imge: pip install yn

In [2]:
# Install this version of pdfminer to solve the "TypeError: '<' not supported between instances of 'tuple' and 'int'" issue.
# https://github.com/pdfminer/pdfminer.six/pull/134
# !pip install git+git://github.com/pdfminer/pdfminer.six.git@1ea9446bd64ea038551ad3eef55b328791a5bb6b

In [3]:
from pdf2image import convert_from_path
import tesserocr
import glob
import time

In [4]:
import threading

In [5]:
import multiprocessing
import time
import logging

logging.basicConfig(filename='./logs/imf-pdf2txt-output-imf-managed-tika-pdfminer-rev2.log', format='%(asctime)s - %(levelname)s - %(message)s', level=logging.ERROR)


In [6]:
logger = logging.getLogger('imf-pdf2text')

In [7]:
def image_to_text_wrapper(index, page, process_output_dict):
    process_output_dict[index] = tesserocr.image_to_text(page)


def parallel_image_to_text(pages, page_timeout=None, page_delimiter='\n'):
    process_output_manager = multiprocessing.Manager()
    process_output_dict = process_output_manager.dict()
    
    ################
    for ix, page in enumerate(pages):
        p = multiprocessing.Process(
            target=image_to_text_wrapper,
            name="parallel_image_to_text",
            args=(ix, page, process_output_dict)
        )
        p.start()

        # Maximum of `page_timeout` second parse time per page.
        p.join(page_timeout)

        # If thread is active
        if p.is_alive():
            # Terminate if the process is still unfinished after the timeout.
            p.terminate()
            p.join()

    text = page_delimiter.join(
        [process_output_dict[i] for i in sorted(process_output_dict.keys())]
    )

    return text

Make sure that the Tika server is up before proceeding!!

`java -jar tika-server-1.18.jar`

In [8]:
import os

os.environ['TIKA_SERVER_JAR'] = '0.0.0.0:8000/tika-server-1.18.jar'
os.environ['TIKA_PATH'] = '/R/NLP/SCRIPTS/TIKA/'
os.environ['TIKA_CLIENT_ONLY'] = 'True'

import tika
tika.initVM()

from tika import parser

In [9]:
# http://stanford.edu/~mgorkove/cgi-bin/rpython_tutorials/Using%20Python%20to%20Convert%20PDFs%20to%20Text%20Files.php#4
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import sys
import getopt
import re
from joblib import Parallel, delayed
import os


text_pattern = re.compile('[a-zA-Z]+')
FREE_CORES = 2


# def timed_process_page(page, pagenum, thread_output_dict, thread_status_dict=None):
#     output = StringIO()
#     manager = PDFResourceManager()
#     converter = TextConverter(manager, output, laparams=LAParams())
#     interpreter = PDFPageInterpreter(manager, converter)

#     interpreter.process_page(page)
#     thread_output_dict[pagenum] = output.getvalue()
    
# #     converter.close()
# #     output.close()
    
#     if thread_status_dict is not None:
#         thread_status_dict[pagenum] = 0


# def convert(fname, pages=None, page_delimiter='\n', page_timeout=4, verbose=False, threads=32):
#     # Note: pages that are set horizontally are processed quite slowly by this converter.
#     # Example: '../data/pdf/00dcaefff8a5d12bccd5061980394da3401ae713.pdf' pages 34-38.
#     # A timed processing is used to skip these pages.
    
#     if not pages:
#         pagenums = set()
#     else:
#         pagenums = set(pages)

#     thread_output_dict = {}
#     thread_status_dict = {}
    
#     output = StringIO()
#     manager = PDFResourceManager()
#     converter = TextConverter(manager, output, laparams=LAParams())
#     interpreter = PDFPageInterpreter(manager, converter)
    
#     infile = open(fname, 'rb')
#     pages = list(PDFPage.get_pages(infile, pagenums))
#     logger.error(f'Total pages in doc {fname} :: {len(pages)}')

#     pages_rotate = []

#     batch = {}
#     pagenum = 0

#     while True:
#         try:
#             if len(batch) < threads:
#                 page = pages.pop(0)
#                 pagenum += 1
#                 p = threading.Thread(target=timed_process_page, name="timed_process_page", args=(page, pagenum, thread_output_dict, thread_status_dict))
#                 pages_rotate.append(page)
#                 p.start()
#                 batch[pagenum] = p
                
#             completed_data = [cpagenum for cpagenum in thread_status_dict.keys()]

#             for cpagenum in completed_data:
#                 status = process_status_dict.pop(cpagenum)
#                 pk = batch.pop(cpagenum)
#                 pk.join()
                
#                 page = pages.pop(0)
#                 pagenum += 1
#                 p = threading.Thread(target=timed_process_page, name="timed_process_page", args=(page, pagenum, thread_output_dict, thread_status_dict))
#                 pages_rotate.append(page)
#                 p.start()
#                 batch[pagenum] = p
#         except:
#             keys = list(batch.keys())
#             for cpagenum in keys:
#                 p = batch.pop(cpagenum)
#                 p.join()  # timeout=page_timeout)
                
#             break

#     # Cleanup just in case... 
#     keys = list(batch.keys())
#     for cpagenum in keys:
#         p = batch.pop(cpagenum)
#         p.join()  # timeout=page_timeout)    


#     #################################################################
#     ##### Try reprocessing missed pages due to possibly rotated orientation.... 
#     #################################################################
    
#     keys = set(thread_output_dict.keys())

#     batch = {}
#     pagenum = 0

#     while True:
#         try:
#             if len(batch) < threads:
#                 page = pages_rotate.pop(0)
#                 pagenum += 1
#                 if pagenum in keys:
#                     continue
                    
#                 p = threading.Thread(target=timed_process_page, name="timed_process_page", args=(page, pagenum, thread_output_dict, thread_status_dict))
#                 p.start()
#                 batch[pagenum] = p
                
#             completed_data = [cpagenum for cpagenum in thread_status_dict.keys()]

#             for cpagenum in completed_data:
#                 status = process_status_dict.pop(cpagenum)
#                 pk = batch.pop(cpagenum)
#                 pk.join()
                
#                 page = pages_rotate.pop(0)
#                 pagenum += 1
#                 if pagenum in keys:
#                     continue

#                 p = threading.Thread(target=timed_process_page, name="timed_process_page", args=(page, pagenum, thread_output_dict, thread_status_dict))
#                 p.start()
#                 batch[pagenum] = p
#         except:
#             keys = list(batch.keys())
#             for cpagenum in keys:
#                 p = batch.pop(cpagenum)
#                 p.join()  # timeout=page_timeout)
                
#             break

#     # Cleanup just in case... 
#     keys = list(batch.keys())
#     for cpagenum in keys:
#         p = batch.pop(cpagenum)
#         p.join()  # timeout=page_timeout)    

#     infile.close()
#     converter.close()
#     output.close()
    
#     if len(thread_output_dict) != pagenum:
#         logger.error(f'WARNING: {fname} incomplete. Only {len(thread_output_dict)} / {pagenum} pages parsed!')

#     text = page_delimiter.join([thread_output_dict[i] for i in sorted(thread_output_dict.keys())])

#     return text



def timed_process_page(page, pagenum, process_output_dict):
    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    interpreter.process_page(page)
    process_output_dict[pagenum] = output.getvalue()

    
def convert_tika(fname):
    parsedPDF = parser.from_file(fname)
    return parsedPDF.get('content', '')


def convert(fname, pages=None, page_delimiter='\n', page_timeout=4, verbose=False):
    # Note: pages that are set horizontally are processed quite slowly by this converter.
    # Example: '../data/pdf/00dcaefff8a5d12bccd5061980394da3401ae713.pdf' pages 34-38.
    # A timed processing is used to skip these pages.
    
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    process_output_manager = multiprocessing.Manager()
    process_output_dict = process_output_manager.dict()

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    processes = []
    
    infile = open(fname, 'rb')
    pages = list(PDFPage.get_pages(infile, pagenums))
    logger.error(f'Total pages in doc {fname} :: {len(pages)}')
    for pagenum, page in enumerate(pages, 1):
        if verbose:
            print(f'{fname}: page {pagenum}')
        # p = multiprocessing.Process(target=timed_process_page, name="timed_process_page", args=(interpreter, page, pagenum, process_output_dict, output))
        p = threading.Thread(target=timed_process_page, name="timed_process_page", args=(page, pagenum, process_output_dict))
        p.start()
        
        processes.append(p)

    for p in processes:
        # Maximum of `page_timeout` second parse time per page.
        p.join(page_timeout)

#         # If thread is active
#         if p.is_alive():
#             # Terminate if the process is still unfinished after the timeout.
#             p.terminate()
#             p.join()

    processes = []
    
    keys = set(process_output_dict.keys())

    # Rotate to recover the landscape pages that are quite slow to parse when not properly oriented.
    for pagenum, page in enumerate(pages, 1):
        if pagenum in keys:
            continue

        page.rotate = 90
        p = threading.Thread(target=timed_process_page, name="timed_process_page", args=(page, pagenum, process_output_dict))
        p.start()
        processes.append(p)
        
    for p in processes:
        p.join(page_timeout)
    
    infile.close()
    converter.close()
    output.close()
    
    if len(process_output_dict) != pagenum:
        logger.error(f'WARNING: {fname} incomplete. Only {len(process_output_dict)} / {pagenum} pages parsed!')

    text = page_delimiter.join([process_output_dict[i] for i in sorted(process_output_dict.keys())])

    return text


def convert_image(fname, thread_count=20, page_delimiter='\n', dpi=200, fmt='ppm', parallel_im2txt=True, cpu_count=None):
    pages = convert_from_path(fname, fmt=fmt, dpi=dpi, thread_count=thread_count)

    # print('Completed comvert_from_path...')
    cpu_count = cpu_count if cpu_count is not None else (os.cpu_count() - FREE_CORES)
    doc = ''
    
    if not parallel_im2txt:
        for p in pages:
            doc += tesserocr.image_to_text(p) + page_delimiter
    else:
        doc = parallel_image_to_text(pages, page_timeout=10, page_delimiter='\n')
#         page_txt = Parallel(n_jobs=cpu_count, backend='threading')(
#             delayed(image_to_text_wrapper)(ix, p) for ix, p in enumerate(pages)
#         )
        
#         page_txt = [p for ix, p in sorted(page_txt)]
#         doc = page_delimiter.join(page_txt)
        
    return doc


def auto_convert(fname, min_valid_chars=1000, thread_count=20, use_tika=True):
    # Convider 1000 chars or ~1kb as the acceptable file size. (min_valid_chars=1000)
    pdf_type = 'text'
    
    if use_tika:
        try:
            text = convert_tika(fname)
            pdf_type = 'text-tika'
        except:
            # Fallback to pdfminer if tika fails.
            logger.error(f'Method tika raised an exception for {fname}! Using pdfminer...')
            text = convert(fname)
            pdf_type = 'text-pdfminer'
    else:
        text = convert(fname)
        pdf_type = 'text-pdfminer'

    if len(text_pattern.findall(text)) < min_valid_chars:
        text = convert_image(fname, thread_count=thread_count, parallel_im2txt=False)
        pdf_type = 'image'
    
    return text, pdf_type


def convert_and_dump(input_fname, txt_dir, min_valid_chars=1000, replace_existing=False):
    logger.error(f'Processing {os.path.basename(input_fname)}...')
    output_fname = os.path.join(txt_dir, 'imf_' + os.path.basename(input_fname).replace('.pdf', '') + '.txt')
    
    if os.path.isfile(output_fname) and (not replace_existing):
        logger.error(f'Document {os.path.basename(input_fname)} already processed...')
        return 0
    
    try:
        text, pdf_type = auto_convert(input_fname, min_valid_chars=min_valid_chars)
    except Exception as e:
        logger.error(f'Error processing ({e.args[0]}): {os.path.basename(input_fname)}...')
        return 1
    
    with open(output_fname, 'w') as fl:
        fl.write(text)
        
    logger.error(f'Successfully converted: {os.path.basename(input_fname)} from {pdf_type} pdf.')
    
    return 0
    

def convert_pdfs_in_dir(pdf_dir, txt_dir, parallel=True, threads=None):
    if not parallel:
        for input_fname in glob.iglob(os.path.join(pdf_dir, '*.pdf')):
            start = time.time()

            convert_and_dump(input_fname, txt_dir)

            print(f'Finished processing {fname} with {len(text)} characters after {time.time() - start:0.2f}')

    else:
        cpu_counts = threads if threads is not None else (os.cpu_count() - FREE_CORES)
        
        Parallel(n_jobs=cpu_counts, backend='threading')(
            delayed(convert_and_dump)(input_fname, txt_dir) for input_fname in glob.iglob(
                os.path.join(pdf_dir, '*.pdf')
            )
        )


In [10]:
pdf_dir = '/R/Corpus/IMF/pdf/'
txt_dir = '/R/Corpus/IMF/tika_imf_pdf_txt'

In [11]:
# import os

fname = '/R/Corpus/IMF/pdf_txt/f058c865ecdde4845f176c860f0e7c6f46bae242.pdf.txt' # 000dc1fcdc7b36b8014131971909b6b4952eb517.pdf'
os.path.getsize(fname)

1481

In [12]:
def managed_convert_and_dump(input_fname, txt_dir, process_status_dict, min_valid_chars=1000, replace_existing=False):
    convert_and_dump(input_fname, txt_dir, min_valid_chars, replace_existing)
    process_status_dict[input_fname] = 0
    
    
def is_outfile_present(fname, txt_dir):
    ofname = os.path.join(txt_dir, 'imf_' + os.path.basename(fname).replace('.pdf', '') + '.txt')
    return os.path.isfile(ofname)
    
    
def managed_dynamic_main(pdf_dir, txt_dir, workers=24, file_ids=None, min_valid_chars=1000, replace_existing=False):
    process_status_manager = multiprocessing.Manager()
    process_status_dict = process_status_manager.dict()

    print('Sorting files by size...')
    
    if file_ids is not None:
        fnames = [os.path.join(pdf_dir, f"{fid.replace('imf_', '')}.pdf") for fid in file_ids]
    else:
        fnames = glob.iglob(os.path.join(pdf_dir, '*.pdf'))
    fnames = sorted(fnames, key=lambda x: os.path.getsize(x))
    print('Start processing pdfs...')
    
    batch = {}
    while True:
        try:
            if len(batch) < workers:
                fname = fnames.pop(0)
                if is_outfile_present(fname, txt_dir) and (not replace_existing):
                    logger.error(f'Document {os.path.basename(fname)} already processed...')
                    continue
                    
                p = multiprocessing.Process(target=managed_convert_and_dump, args=(fname, txt_dir, process_status_dict, min_valid_chars, replace_existing))
                p.start()
                batch[fname] = p
                
            completed_data = [cfname for cfname in process_status_dict.keys()]

            for cfname in completed_data:
                status = process_status_dict.pop(cfname)
                pk = batch.pop(cfname)
                pk.join()
                
                if pk.is_alive():
                    pk.terminate()
                    pk.join()

                fname = fnames.pop(0)
                if is_outfile_present(fname, txt_dir) and (not replace_existing):
                    logger.error(f'Document {os.path.basename(fname)} already processed...')
                    continue
                    
                p = multiprocessing.Process(target=managed_convert_and_dump, args=(fname, txt_dir, process_status_dict, min_valid_chars, replace_existing))
                p.start()
                batch[fname] = p
        except:
            for fname in batch:
                p = batch[fname]
                p.join()
                
                if p.is_alive():
                    p.terminate()
                    p.join()
   
            break

    # Cleanup just in case... 
    for fname in batch:
        p = batch[fname]
        p.join()

        if p.is_alive():
            p.terminate()
            p.join()


In [13]:
%%time
# Not well parsed by commandline tika?: ccb71ccee9e1cdbd4310591a12f9b7227e1199c2 :(
# Not well parsed by tika and tesseract: b315d239370d9b6780982fffd4bc924895ff83f9.pdf :(
if __name__ == '__main__':
    managed_dynamic_main(pdf_dir, txt_dir, workers=48)

Sorting files by size...
Start processing pdfs...
CPU times: user 2.05 s, sys: 1.43 s, total: 3.48 s
Wall time: 55.5 s


In [ ]:
# logging.basicConfig(filename='./logs/imf-pdf2txt-output-imf-managed-tika-pdfminer-review.log', format='%(asctime)s - %(levelname)s - %(message)s', level=logging.ERROR)
# logger = logging.getLogger('imf-review')

In [14]:
%%time
MIN_VALID_CHARS = 11000

file_ids = []

for forig_name in glob.iglob('../CORPUS/IMF/TXT_ORIG/imf_*.txt'):
    if os.path.getsize(forig_name) < MIN_VALID_CHARS:
        file_ids.append(os.path.basename(forig_name).replace('.txt', ''))
# os.path.getsize('../CORPUS/IMF/TXT_ORIG/imf_95193976af5c2fc556fd99f6dab80070f938b3fe.txt')

print(len(file_ids))

359
CPU times: user 385 ms, sys: 462 ms, total: 847 ms
Wall time: 24.7 s


In [15]:
%%time
# Rerun for possibly false-negative images
# Not well parsed by commandline tika?: ccb71ccee9e1cdbd4310591a12f9b7227e1199c2 :(
# Not well parsed by tika and tesseract: b315d239370d9b6780982fffd4bc924895ff83f9.pdf :(
if __name__ == '__main__':
    managed_dynamic_main(pdf_dir, txt_dir, workers=48, file_ids=file_ids, min_valid_chars=MIN_VALID_CHARS, replace_existing=True)

Sorting files by size...
Start processing pdfs...
CPU times: user 3min 17s, sys: 42.5 s, total: 3min 59s
Wall time: 56min 52s


In [ ]:
if __name__ == '__main__':
    tasks = []
    workers = 96
    print('Sorting files by size...')
    fnames = sorted(glob.iglob(os.path.join(pdf_dir, '*.pdf')), key=lambda x: os.path.getsize(x))
    print('Start processing pdfs...')
    
    for fname in fnames:
        ofname = os.path.join(txt_dir, 'imf_' + os.path.basename(fname).replace('.pdf', '') + '.txt')
        if os.path.isfile(ofname):
            logger.error(f'Document {os.path.basename(fname)} already processed...')
            continue
            
        if len(tasks) < workers:
            tasks.append(fname)
        else:
            processes = []
            for input_fname in tasks:
                p = multiprocessing.Process(target=convert_and_dump, args=(input_fname, txt_dir))
                p.start()
                processes.append(p)

            for p in processes:
                p.join()
                
            tasks = []
    if tasks:
        processes = []
        for input_fname in tasks:
            p = multiprocessing.Process(target=convert_and_dump, args=(input_fname, txt_dir))
            p.start()
            processes.append(p)

        for p in processes:
            p.join()

        tasks = []

# Rerun for originally faulty parsing

### Original size > 500kb and parsed size < 50kb

In [11]:
max_parsed_size = 50000
min_original_size = 500000

if __name__ == '__main__':
    tasks = []
    workers = 96
    print('Sorting files by size...')
    fnames = sorted(glob.iglob(os.path.join(txt_dir, '*.pdf.txt')), key=lambda x: os.path.getsize(x))
    print('Start processing pdfs...')
    
    for fname in fnames:
        ofname = fname
        fname = ofname.replace('.txt', '')
        fname = os.path.join(pdf_dir, os.path.basename(fname))
        
        fsize = os.path.getsize(fname)
        if os.path.isfile(ofname):
            ofsize = os.path.getsize(ofname)
            
            if (fsize > min_original_size) and (ofsize < max_parsed_size):
                pass
            else:
                logger.error(f'Document {os.path.basename(fname)} already processed...')
                continue
            
        if len(tasks) < workers:
            tasks.append(fname)
        else:
            processes = []
            for input_fname in tasks:
                p = multiprocessing.Process(target=convert_and_dump, args=(input_fname, txt_dir))
                p.start()
                processes.append(p)

            for p in processes:
                p.join()
                
            tasks = []
    if tasks:
        processes = []
        for input_fname in tasks:
            p = multiprocessing.Process(target=convert_and_dump, args=(input_fname, txt_dir))
            p.start()
            processes.append(p)

        for p in processes:
            p.join()

        tasks = []

Sorting files by size...
Start processing pdfs...


In [ ]:
# 13954 9/14/2018 3:36 AM
# 14020 9/14/2018 6:46 AM

In [13]:
f = 'e1c91148692198e96e7020b9e5919aa935308169.pdf'
# f = '6497ba5f6ea18ca5c3ded8ef947510f562a3db89.pdf'
f = '4c421ce6c630e5e77550d319e5cbd71f1c1c4842.pdf'
fname = os.path.join('/R/Corpus/IMF/pdf/', f)  # 000dc1fcdc7b36b8014131971909b6b4952eb517.pdf'

# t = auto_convert(fname)
# t = convert(fname, page_timeout=10, verbose=True)  # , threads=1000)

In [16]:
t = auto_convert(fname)

In [18]:
len(t[0])

7977

In [19]:
print(t[0])

@2003 International Monetary F und March 2003
IMF Country Repon No. 03/85

Uganda: Report on Observance of Standards and Codes—
Fiscal Transparency Module—Update

This update to the Report on the Observance of Standards and Codes on Fiscal Transparency for
Uganda was prepaIed by a staff team of the Intemational Monetary Fund as background documentation
for the periodic consultation with the member country It is based on the information available at the time
it was completed on J anuary 24, 2003. The views expressed in this docmnent are those of the staff team
and do not necessan'ly reﬂect the views of the govemmem of Uganda or the Executive Board of the IMF.

The policy of publication of staff reports and other docmnents by the IMF allows for the deletion of
market-sensitive information.

To assist the IMF in evaluating the publication policy, reader comments are invited and may be

sent by e-mail to Eublicationgolic!@imf.urg.

Copies of this report are available to the public from

In

In [35]:
len(t)

48575

In [12]:

def convert_image(fname, thread_count=20, page_delimiter='\n', dpi=200, fmt='ppm', parallel_im2txt=True, cpu_count=None):
    pages = convert_from_path(fname, fmt=fmt, dpi=dpi, thread_count=thread_count)

    # print('Completed comvert_from_path...')
    cpu_count = cpu_count if cpu_count is not None else (os.cpu_count() - FREE_CORES)
    doc = ''
    
    if not parallel_im2txt:
        for p in pages:
            doc += tesserocr.image_to_text(p) + page_delimiter
    else:
        doc = parallel_image_to_text(pages, page_timeout=10, page_delimiter='\n')
#         page_txt = Parallel(n_jobs=cpu_count, backend='threading')(
#             delayed(image_to_text_wrapper)(ix, p) for ix, p in enumerate(pages)
#         )
        
#         page_txt = [p for ix, p in sorted(page_txt)]
#         doc = page_delimiter.join(page_txt)
        
    return doc


In [16]:
%%time
t = convert_image(fname, thread_count=100, parallel_im2txt=False)

CPU times: user 26.6 s, sys: 91 ms, total: 26.7 s
Wall time: 27.6 s


In [14]:
%%time
t = convert_image(fname, parallel_im2txt=False)

CPU times: user 26.4 s, sys: 138 ms, total: 26.6 s
Wall time: 27.5 s


In [15]:
%%time
t = convert_image(fname, parallel_im2txt=True)

CPU times: user 270 ms, sys: 113 ms, total: 383 ms
Wall time: 27.8 s


In [19]:
len(t)

42594